# 🚀 Flujo Completo: Dataset → Entrenamiento → Clasificación

Este notebook integra los 3 módulos principales del proyecto:
1. **DatasetGenerator**: Genera datos de letras con distorsión
2. **MLP**: Red neuronal que aprende a clasificar
3. **ClasificadorLetras**: Interfaz para usar y evaluar el modelo

---

## 📦 Paso 1: Importar Librerías y Módulos

In [ ]:
# Librerías estándar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Módulos del proyecto
from generador_dataset import DatasetGenerator
from mlp import MLP
from clasificador import ClasificadorLetras

print("✅ Librerías y módulos importados correctamente")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")

---
# 🏭 PARTE 1: Generación de Datos (DatasetGenerator)

Vamos a generar un dataset de 500 ejemplos con distorsiones específicas.

## 🎯 Paso 2: Crear el Generador de Datos

In [ ]:
# Crear instancia del generador
generador = DatasetGenerator()

print("✅ DatasetGenerator creado")
print(f"   - Letras disponibles: {generador.letras}")
print(f"   - Directorio base: {generador.base_path}")

## 📊 Paso 3: Generar Datasets (Originales + Distorsionados)

In [ ]:
# Configuración
cantidad = 500
distorsiones = [1, 3, 5]  # Porcentajes de distorsión

print(f"🎯 Generando dataset de {cantidad} ejemplos...")
print(f"   Distorsiones: {distorsiones}%")
print("="*60)

# Generar datos originales
print("\n1️⃣ Generando datos originales...")
generador.generar_data_letras(cantidad)

# Generar datos con distorsiones específicas
print("\n2️⃣ Generando datos con distorsiones específicas...")
generador.generar_data_con_distorsiones_especificas(
    cant=cantidad,
    distorsiones=distorsiones,
    mezclar=False  # No mezclar para poder comparar después
)

print("\n" + "="*60)
print("✅ Datasets generados exitosamente!")

## 📂 Paso 4: Cargar los Datos Generados

In [ ]:
def leer_dataset(cantidad, tipo='originales'):
    """Lee un dataset desde CSV y separa patrones de etiquetas"""
    file_path = os.path.join('data', tipo, str(cantidad), 'letras.csv')
    df = pd.read_csv(file_path, sep=';', header=None)
    X = df.iloc[:, :100].values  # Primeros 100 columnas (patrón 10x10)
    y = df.iloc[:, 100:].values   # Últimas 3 columnas (etiquetas one-hot)
    return X, y

# Cargar datos
print("📥 Cargando datasets...\n")
X_orig, y_orig = leer_dataset(cantidad, 'originales')
print(f"✅ Originales: X={X_orig.shape}, y={y_orig.shape}")

X_dist, y_dist = leer_dataset(cantidad, 'distorsionadas')
print(f"✅ Distorsionados: X={X_dist.shape}, y={y_dist.shape}")

print(f"\n📊 Total: {len(X_dist)} patrones listos para entrenar")

## 🔍 Paso 5: Visualizar Algunos Patrones

In [ ]:
def visualizar_ejemplos(X_orig, y_orig, X_dist, num_ejemplos=6):
    """Visualiza ejemplos originales vs distorsionados"""
    letras_map = {0: 'B', 1: 'D', 2: 'F'}
    fig, axes = plt.subplots(2, num_ejemplos, figsize=(num_ejemplos*2.5, 5))
    
    for i in range(num_ejemplos):
        letra = letras_map[np.argmax(y_orig[i])]
        
        # Original
        axes[0, i].imshow(X_orig[i].reshape(10, 10), cmap='binary', interpolation='nearest')
        axes[0, i].set_title(f'Original: {letra}', fontsize=10, fontweight='bold')
        axes[0, i].axis('off')
        
        # Distorsionado
        axes[1, i].imshow(X_dist[i].reshape(10, 10), cmap='binary', interpolation='nearest')
        distorsion = np.sum(X_dist[i] != X_orig[i]) / len(X_orig[i]) * 100
        axes[1, i].set_title(f'Dist: {distorsion:.1f}%', fontsize=10)
        axes[1, i].axis('off')
    
    axes[0, 0].set_ylabel('Originales', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Distorsionados', fontsize=12, fontweight='bold')
    plt.suptitle('Comparación: Originales vs Distorsionados', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualizar
visualizar_ejemplos(X_orig, y_orig, X_dist, num_ejemplos=6)

---
# 🧠 PARTE 2: Entrenamiento del Modelo (MLP)

Ahora vamos a entrenar una red neuronal para que aprenda a clasificar las letras.

## 🔄 Paso 6: Preparar Datos para Entrenamiento

In [ ]:
# Mezclar los datos (importante para el entrenamiento)
print("🔀 Mezclando datos para entrenamiento...")

indices = np.arange(len(X_dist))
np.random.shuffle(indices)

X_train = X_dist[indices]
y_train = y_dist[indices]

print(f"✅ Datos mezclados: {len(X_train)} ejemplos listos")
print(f"   - Forma X_train: {X_train.shape}")
print(f"   - Forma y_train: {y_train.shape}")

## 🏗️ Paso 7: Crear y Configurar el MLP

In [ ]:
# Configuración del modelo
arquitectura = [100, 10, 3]  # 100 entrada, 10 oculta, 3 salida
funciones = ['sigmoidal', 'sigmoidal']  # Función de activación para cada capa
learning_rate = 0.1
momentum = 0.9

print("🏗️ Creando MLP...")
print(f"   - Arquitectura: {arquitectura[0]} → {arquitectura[1]} → {arquitectura[2]}")
print(f"   - Funciones de activación: {funciones}")
print(f"   - Learning rate: {learning_rate}")
print(f"   - Momentum: {momentum}")
print()

# Crear el modelo
mlp = MLP(
    arquitectura=arquitectura,
    funciones_activacion=funciones,
    learning_rate=learning_rate,
    momentum=momentum
)

print("✅ MLP creado exitosamente!")

## 🎯 Paso 8: Entrenar el Modelo

In [ ]:
# Configuración del entrenamiento
epochs = 1000

print("🎯 Entrenando el modelo...")
print(f"   - Épocas: {epochs}")
print(f"   - Ejemplos de entrenamiento: {len(X_train)}")
print("="*60)

# Entrenar
historial = mlp.entrenar(
    X_train,
    y_train,
    epochs=epochs,
    verbose=True
)

print("="*60)
print(f"✅ Entrenamiento completado!")
print(f"   - Error inicial: {historial[0]:.6f}")
print(f"   - Error final: {historial[-1]:.6f}")
print(f"   - Mejora: {((historial[0] - historial[-1]) / historial[0] * 100):.2f}%")

## 📈 Paso 9: Visualizar Curva de Aprendizaje

In [ ]:
# Graficar evolución del error
plt.figure(figsize=(12, 5))

# Gráfica 1: Error completo
plt.subplot(1, 2, 1)
plt.plot(historial, linewidth=2, color='blue')
plt.xlabel('Época', fontsize=12, fontweight='bold')
plt.ylabel('Error (MSE)', fontsize=12, fontweight='bold')
plt.title('Curva de Aprendizaje Completa', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Gráfica 2: Últimas 200 épocas (zoom)
plt.subplot(1, 2, 2)
plt.plot(historial[-200:], linewidth=2, color='green')
plt.xlabel('Época (últimas 200)', fontsize=12, fontweight='bold')
plt.ylabel('Error (MSE)', fontsize=12, fontweight='bold')
plt.title('Convergencia Final', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 El modelo convergió correctamente")
print(f"   Error final: {historial[-1]:.6f}")

---
# 🎛️ PARTE 3: Uso y Evaluación (ClasificadorLetras)

Ahora vamos a usar el clasificador para evaluar y visualizar el rendimiento del modelo.

## 🎨 Paso 10: Crear el Clasificador

In [ ]:
# Crear clasificador con el modelo entrenado
clasificador = ClasificadorLetras(mlp)

print("✅ Clasificador creado con el modelo MLP entrenado")
print("   Listo para clasificar patrones y evaluar rendimiento")

## 🧪 Paso 11: Evaluar Precisión en Datos de Entrenamiento

In [ ]:
# Evaluar precisión
print("📊 Evaluando precisión del modelo...\n")

letras_map = {0: 'B', 1: 'D', 2: 'F'}
aciertos = 0
aciertos_por_letra = {'B': 0, 'D': 0, 'F': 0}
total_por_letra = {'B': 0, 'D': 0, 'F': 0}

for i in range(len(X_dist)):
    prediccion = clasificador.clasificar_patron(X_dist[i])
    letra_real = letras_map[np.argmax(y_dist[i])]
    
    total_por_letra[letra_real] += 1
    
    if prediccion == letra_real:
        aciertos += 1
        aciertos_por_letra[letra_real] += 1

precision_global = (aciertos / len(X_dist)) * 100

print(f"✅ Precisión Global: {precision_global:.2f}%")
print(f"   Aciertos: {aciertos}/{len(X_dist)}\n")

print("📊 Precisión por Letra:")
for letra in ['B', 'D', 'F']:
    prec = (aciertos_por_letra[letra] / total_por_letra[letra]) * 100
    print(f"   {letra}: {prec:.2f}% ({aciertos_por_letra[letra]}/{total_por_letra[letra]})")

## 🔍 Paso 12: Visualizar Predicciones

In [ ]:
# Visualizar algunas predicciones
num_ejemplos = 12
indices_aleatorios = np.random.choice(len(X_dist), num_ejemplos, replace=False)

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for idx, i in enumerate(indices_aleatorios):
    prediccion = clasificador.clasificar_patron(X_dist[i])
    letra_real = letras_map[np.argmax(y_dist[i])]
    distorsion = np.sum(X_dist[i] != X_orig[i]) / len(X_orig[i]) * 100
    
    # Determinar si es correcto o incorrecto
    correcto = "✅" if prediccion == letra_real else "❌"
    color = 'green' if prediccion == letra_real else 'red'
    
    axes[idx].imshow(X_dist[i].reshape(10, 10), cmap='binary', interpolation='nearest')
    axes[idx].set_title(f'{correcto} Real: {letra_real} | Pred: {prediccion}\nDist: {distorsion:.1f}%', 
                        fontsize=10, color=color, fontweight='bold')
    axes[idx].axis('off')

plt.suptitle('Predicciones del Modelo (Verde=Correcto, Rojo=Error)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🎯 Paso 13: Probar Robustez ante Distorsiones

In [ ]:
# Probar con diferentes niveles de distorsión
print("🧪 Probando robustez del modelo con diferentes distorsiones...")
print("="*60)

distorsiones_prueba = [0, 1, 3, 5, 10, 15, 20, 25, 30]
num_pruebas = 20

for letra in ['B', 'D', 'F']:
    print(f"\n📝 Letra {letra}:")
    clasificador.probar_distorsiones(
        letra=letra,
        distorsiones=distorsiones_prueba,
        num_pruebas=num_pruebas
    )

print("\n" + "="*60)

## 📈 Paso 14: Evaluar y Graficar Robustez Completa

In [ ]:
# Evaluar robustez completa
print("📊 Evaluando robustez completa del modelo...\n")

resultados = clasificador.evaluar_robustez(
    distorsiones=[0, 5, 10, 15, 20, 25, 30],
    num_pruebas_por_letra=50
)

print("\n📈 Generando gráfica de robustez...")
print("="*60)

# Extraer datos
distorsiones_test = resultados['distorsiones']
precision_B = resultados['precision_B']
precision_D = resultados['precision_D']
precision_F = resultados['precision_F']
precision_global = resultados['precision_global']

# Crear gráfica
plt.figure(figsize=(12, 7))
plt.plot(distorsiones_test, precision_B, 'o-', label='Letra B', linewidth=2, markersize=8)
plt.plot(distorsiones_test, precision_D, 's-', label='Letra D', linewidth=2, markersize=8)
plt.plot(distorsiones_test, precision_F, '^-', label='Letra F', linewidth=2, markersize=8)
plt.plot(distorsiones_test, precision_global, 'D-', label='Global', linewidth=3, markersize=8, color='black')

plt.xlabel('Distorsión (%)', fontsize=12, fontweight='bold')
plt.ylabel('Precisión (%)', fontsize=12, fontweight='bold')
plt.title('Robustez del Modelo ante Distorsiones', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim([0, 105])
plt.tight_layout()
plt.show()

print("="*60)
print("✅ Evaluación de robustez completada!")

## 💾 Paso 15: Guardar el Modelo Entrenado

In [ ]:
# Guardar modelo
nombre_modelo = 'modelo_completo_500_ejemplos.json'

print(f"💾 Guardando modelo...")
clasificador.guardar_modelo(nombre_modelo)

print(f"✅ Modelo guardado como: {nombre_modelo}")
print(f"\n📝 Para cargar el modelo más tarde:")
print(f"   clasificador.cargar_modelo('{nombre_modelo}')")

---
## 🎉 Resumen Final

### ✅ Lo que hicimos:

**1. DatasetGenerator:**
- ✅ Generamos 500 patrones originales de letras B, D, F
- ✅ Aplicamos distorsiones específicas (1%, 3%, 5%)
- ✅ Guardamos datos en formato CSV

**2. MLP:**
- ✅ Creamos red neuronal con arquitectura 100 → 10 → 3
- ✅ Entrenamos con 1000 épocas
- ✅ Redujimos el error significativamente

**3. ClasificadorLetras:**
- ✅ Evaluamos precisión del modelo
- ✅ Visualizamos predicciones
- ✅ Probamos robustez ante distorsiones
- ✅ Guardamos el modelo para uso futuro

### 📊 Resultados Obtenidos:
- **Precisión en entrenamiento:** Ver Paso 11
- **Error final:** Ver Paso 8
- **Robustez:** Ver gráfica en Paso 14

### 🔧 Próximos Pasos:
- Experimentar con diferentes arquitecturas (más neuronas ocultas)
- Probar con más datos (1000 ejemplos)
- Ajustar hiperparámetros (learning_rate, momentum)
- Comparar diferentes configuraciones